In [6]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import pandas as pd
import numpy as np
import random
import time

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from highway_env_modified.highway_env.envs.random_flow import RandomFlowEnv
import torch.nn as nn
from tqdm import tqdm

In [7]:
df_dataset = pd.read_csv("collision_datasets_see_behind_true_absolute_false/position_vector_sorted_simulation_policy_frequency_fixed_crash_dataset.csv", index_col = 0)
df_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.0,0.117533,0.640909,0.277620,0.129491,0.906264,0.422712,1.0,0.004850,-0.015497,...,-0.492177,-0.870495,1.0,0.032808,-0.156332,0.052694,0.071150,0.854680,0.519155,1.0
1,1.0,0.150970,0.020293,0.295720,-0.092583,0.954323,-0.298777,1.0,-0.008965,-0.001376,...,-0.426482,-0.904496,1.0,-0.056187,0.116976,0.076248,0.145663,0.989971,0.141269,1.0
2,1.0,0.141395,0.475949,0.269381,0.121366,0.911738,0.410772,1.0,-0.006281,-0.076622,...,0.850719,-0.525621,1.0,-0.058192,0.113400,-0.453947,-0.439538,-0.501770,-0.865001,0.0
3,1.0,0.149159,0.470361,0.279619,0.019703,0.997527,0.070289,1.0,-0.010033,-0.037594,...,0.850719,-0.525621,1.0,-0.070878,0.083635,-0.460371,-0.331302,-0.501770,-0.865001,1.0
4,1.0,0.163350,0.728161,0.302070,0.140896,0.906264,0.422712,1.0,0.001406,-0.055891,...,0.948396,0.317088,1.0,-0.034097,0.144628,0.032101,-0.132510,0.999685,0.025085,1.0


In [8]:
df_dataset.tail()

C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
972109,1.0,0.289963,0.032716,0.229917,-0.244369,0.685243,-0.728315,1.0,0.020165,-0.056996,...,0.911782,-0.410675,1.0,-0.124498,0.177988,-0.095121,0.131877,0.767768,-0.640728,1.0
972110,1.0,0.179758,0.688791,0.278522,-0.120845,0.917373,-0.398029,1.0,-0.015008,0.075524,...,0.878508,0.477728,1.0,-0.029164,-0.192622,0.037082,0.310227,0.857469,0.514535,1.0
972111,1.0,0.202732,0.324573,0.265851,0.124002,0.906264,0.422712,1.0,-0.002844,-0.026274,...,0.999001,-0.044685,1.0,-0.016213,-0.290175,0.091883,-0.199384,0.978511,-0.206193,1.0
972112,1.0,0.173815,0.052664,0.282249,0.095795,0.946946,0.321394,1.0,0.017648,0.058754,...,-0.438151,-0.898901,1.0,-0.035941,0.597319,0.043977,0.036104,0.927090,0.374840,0.0
972113,1.0,0.180865,0.058572,0.270096,0.091671,0.946946,0.321394,1.0,0.020411,0.041291,...,-0.438151,-0.898901,1.0,0.014855,0.610038,0.062481,-0.161274,0.978794,-0.204846,1.0


In [9]:
df_dataset.shape

C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(972114, 50)

In [10]:
print("Number of label=1: ", df_dataset[df_dataset.iloc[:,-1] == 1.0].shape[0])
print("Number of label=0: ", df_dataset[df_dataset.iloc[:,-1] == 0.0].shape[0])

Number of label=1:  169357
Number of label=0:  802757


In [11]:
def remove_labels_from_dataset(df_dataset_new, drop_which_label, drop_how_many):
    last_col_index = df_dataset_new.iloc[:,-1].name
    # finding out the indices that have 1 as label
    indices_list = list(df_dataset_new[df_dataset_new[last_col_index] == float(drop_which_label)].index)
    # shuffling the indices
    random.shuffle(indices_list)
    # taking random `drop_how_many` samples where label is 1 out of total samples
    indices_list = indices_list[ : drop_how_many]
    # dropping those samples
    df_dataset_new.drop(indices_list, inplace = True)
    # reset index 
    df_dataset_new.reset_index(drop = True, inplace = True)
    return df_dataset_new

C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
df_dataset_new = remove_labels_from_dataset(df_dataset, drop_which_label = 0.0, drop_how_many = 633400)
df_dataset_new.shape
#df_dataset_new = df_dataset

(338714, 50)

In [13]:
print("Number of label=1: ", df_dataset_new[df_dataset_new.iloc[:,-1] == 1.0].shape[0])
print("Number of label=0: ", df_dataset_new[df_dataset_new.iloc[:,-1] == 0.0].shape[0])

Number of label=1:  169357
Number of label=0:  169357


C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#df_dataset_new.to_csv("collision_datasets_see_behind_true_absolute_false/mini_new_dataset.csv")

In [18]:
X = df_dataset_new.iloc[:,:-1].to_numpy()  # removing the label column and converting to numpy array
y = df_dataset_new.iloc[:,-1].to_numpy()   # extracting the label column and converting to numpy array

In [19]:
X.shape

(338714, 49)

In [20]:
y.shape

(338714,)

In [21]:
#model
rf = RandomForestClassifier(n_estimators=100, class_weight={0: 1, 1: 1}, random_state=50, max_depth=10)
# choice of hyperparams:
# log_2(420) = 8
# square root of 420 = 20
# 7 * 5 = n_features * n_vehicels = 35

# rf = RandomForestClassifier(n_estimators=200, max_depth=35, max_features="sqrt", \
#                             n_jobs=-1, class_weight={0: 1, 1: 1}, \
#                             max_samples=0.7, random_state=100)

In [22]:
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': {0: 1, 1: 1}, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 50, 'verbose': 0, 'warm_start': False}


In [ ]:
start = time.time()
#train
rf.fit(X, y)
end = time.time()
print("time taken to train: ", (end - start))

In [ ]:
train_preds = rf.predict(X)

In [ ]:
#calculate training accuracy
train_acc = accuracy_score(y, train_preds)
train_acc_percent = train_acc * 100
train_error_percent = 100 - train_acc_percent
print("Train Set Error:", round(train_error_percent,3), "%")
print("Train Set Accuracy:", round(train_acc_percent,3), "%")

In [ ]:
prec_score_1 = precision_score(y, train_preds, pos_label = 1)
recall_score_1 =  recall_score(y, train_preds, pos_label = 1)
print("Precision of label=1: ", prec_score_1)
print("Recall of label=1: ", recall_score_1)

In [ ]:
prec_score_0 = precision_score(y, train_preds, pos_label = 0)
recall_score_0 =  recall_score(y, train_preds, pos_label = 0)
print("Precision of label=0: ", prec_score_0)
print("Recall of label=0: ", recall_score_0)

In [3]:
# import joblib
# rf = joblib.load("collision_datasets_see_behind_true_absolute_false\predict_current_time_step\predict_current_time_step_corner_position_vectors_rf_100_history_size_2.joblib")

In [5]:
# creating an instance of the environment
env = RandomFlowEnv()
#env = IntersectionEnv()
env.configure({
    "observation": {
        "type": 'Kinematics',
        "features": ["presence", "x", "y", "vx", "vy", "cos_h", "sin_h"],
        "vehicles_count": 7,
        "absolute": False
    },
    "vehicles_count": 20,
    "screen_width": 520,
    "screen_height": 350,
    "simulation_frequency": 15,  # [Hz]
    "policy_frequency": 15,  # [Hz]
    
})
print(env.config)
print("absolute: ", env.config['observation']['absolute'])
print("see_behind: ", env.observation_type.see_behind)
env.reset()

{'observation': {'type': 'Kinematics', 'features': ['presence', 'x', 'y', 'vx', 'vy', 'cos_h', 'sin_h'], 'vehicles_count': 7, 'absolute': False}, 'action': {'type': 'DiscreteMetaAction'}, 'simulation_frequency': 15, 'policy_frequency': 15, 'other_vehicles_type': 'highway_env.vehicle.random_flow_vehicle.RandomFlowVehicle', 'screen_width': 520, 'screen_height': 350, 'centering_position': [0.3, 0.5], 'scaling': 5.5, 'show_trajectories': False, 'render_agent': True, 'offscreen_rendering': False, 'manual_control': False, 'real_time_rendering': False, 'lanes_count': 12, 'vehicles_count': 20, 'controlled_vehicles': 1, 'initial_lane_id': None, 'duration': 70, 'ego_spacing': 5, 'vehicles_density': 1, 'collision_reward': -1, 'right_lane_reward': 0.1, 'high_speed_reward': 0.4, 'lane_change_reward': 0, 'reward_speed_range': [20, 30], 'offroad_terminal': True}
absolute:  False
see_behind:  True


array([[ 1.00000000e+00,  1.15174115e-01,  5.40921032e-01,
         3.12500000e-01,  0.00000000e+00,  1.00000000e+00,
         2.44929371e-16],
       [ 1.00000000e+00, -1.28595764e-02, -6.38445653e-03,
         4.42347154e-02,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00, -1.80387869e-02,  1.28757756e-03,
         5.79482839e-02,  0.00000000e+00,  1.00000000e+00,
         2.44929371e-16],
       [ 1.00000000e+00, -3.06541007e-02,  9.58112068e-03,
         6.34194911e-02,  0.00000000e+00,  1.00000000e+00,
         2.44929371e-16],
       [ 1.00000000e+00, -3.40291671e-02,  4.31453902e-03,
         7.25917518e-02,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00, -3.46398749e-03, -1.56818032e-01,
         7.69158304e-02,  0.00000000e+00,  1.00000000e+00,
         2.44929371e-16],
       [ 1.00000000e+00,  1.73920323e-03,  1.62734598e-01,
         4.89414707e-02,  0.00000000e+00,  1.00000000e+00,
         0.0000000

In [7]:
def real_to_scaled(x,y):
    '''
    default.vehicle_length = 5
    MAX_SPEED = 40
    number_of_lanes = 12
    width_of_lane = 4
    features_range = [-1,1]
    x_range = [-default.vehicle_length * MAX_SPEED, -default.vehicle_length * MAX_SPEED]
    y_range = [-default.width_of_lane * number_of_lanes, default.width_of_lane * number_of_lanes]
    '''
    scaled_x = (((x + 200) / 400)*2)-1
    scaled_y = (((y + 48) / 96)*2)-1
    return np.array([scaled_x, scaled_y])

def scaled_to_real(scaled_x, scaled_y):
    '''
    default.vehicle_length = 5
    MAX_SPEED = 40
    number_of_lanes = 12
    width_of_lane = 4
    features_range = [-1,1]
    x_range = [-default.vehicle_length * MAX_SPEED, -default.vehicle_length * MAX_SPEED]
    y_range = [-default.width_of_lane * number_of_lanes, default.width_of_lane * number_of_lanes]
    '''
    x = (((scaled_x + 1) * 400) / 2) - 200
    y = (((scaled_y + 1) * 96) / 2) - 48
    return np.array([x, y])

def get_polygon_coordinates(cos_h, position=[0.1,0.1], vehicle_length = 5, vehicle_width = 2):
    '''position is a list of x,y coordinates'''
    
    position = np.array(position)
    position = scaled_to_real(position[0], position[1])
    LENGTH = vehicle_length
    WIDTH = vehicle_width
    heading = np.arccos(np.array(cos_h))
    
    points = np.array([
        [-LENGTH / 2, -WIDTH / 2],
        [-LENGTH / 2, +WIDTH / 2],
        [+LENGTH / 2, +WIDTH / 2],
        [+LENGTH / 2, -WIDTH / 2],
    ]).T
    c, s = np.cos(heading), np.sin(heading)
    rotation = np.array([
        [c, -s],
        [s, c]
    ])
    points = (rotation @ points).T + np.tile(position, (4, 1))
    #array=np.vstack([points, points[0:1]])
    
    first_point = real_to_scaled(points[0][0],points[0][1])
    second_point = real_to_scaled(points[1][0],points[1][1])
    third_point = real_to_scaled(points[2][0],points[2][1])
    fourth_point = real_to_scaled(points[3][0],points[3][1])
    
    return first_point, second_point, third_point, fourth_point

def add_corner_position_vectors(crash_dataset, n_vehicles = 7, n_points = 4):
    
    # naming all the columns
    col_name_list = []
    for i in range(n_vehicles):
      col_name_list = col_name_list + ['presence_'+str(i),'x_'+str(i),'y_'+str(i),'vx_'+str(i),'vy_'+str(i),'cos_h_'+str(i),'sin_h_'+str(i)]
    #col_name_list = col_name_list + ['collision']
    crash_dataset.columns = col_name_list
    
    n_samples = crash_dataset.shape[0]

    for nth_car in tqdm(range(n_vehicles)):
        x_nth_car = np.zeros((n_points, n_samples))
        y_nth_car = np.zeros((n_points, n_samples)) 

        # goes through all the samples for a single vehicle
        for nth_sample in range(n_samples):
            coords = get_polygon_coordinates(crash_dataset['cos_h_'+str(nth_car)][nth_sample], [crash_dataset['x_'+str(nth_car)][nth_sample],\
                                                                              crash_dataset['y_'+str(nth_car)][nth_sample]])
            for nth_point in range(n_points):
                x_nth_car[nth_point][nth_sample] = coords[nth_point][0]
                y_nth_car[nth_point][nth_sample] = coords[nth_point][1] 

        # creating and inserting the heading column
        # this is for a single vehicle
        for nth_point in range(n_points):
            crash_dataset.insert(loc= crash_dataset.columns.get_loc('y_'+str(nth_car)) + 2*nth_point + 1,\
                        column='x_'+str(nth_car)+"_"+str(nth_point+1), value=x_nth_car[nth_point])
            crash_dataset.insert(loc= crash_dataset.columns.get_loc('y_'+str(nth_car)) + 2*nth_point + 2,\
                        column='y_'+str(nth_car)+"_"+str(nth_point+1), value=y_nth_car[nth_point]) 
    
    return crash_dataset.iloc[:].to_numpy()

C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# X_sim = np.array([]) # contains all feature values
# has_crashed = [] # contains crash (1) or no-crash (0) for each time step
# preds_list = []
# inspect_X = np.array([])
# testing_duration = 1000

# start = time.time()
# for step in tqdm(range(testing_duration)):
#     action = None
#     obs, reward, done, info = env.step(action)

#     temp_obs = obs.flatten() # this returns a 1D numpy array [5, 1, 23, 4,...]
#     X_sim = np.expand_dims(temp_obs, axis = 0) # turning dataset into 2D; [[5, 1, 23, 4,...]]
    
#     X_sim = add_corner_position_vectors(pd.DataFrame(X_sim))
#     #X_sim = inserting_col(X_sim)
    
# #     # manually scaling the data since minmaxscaler formula assumes 2D- data; here we are transforming one sample
# #     # at each time step
# #     #X_scaled = scaler.fit_transform(X_sim)
# #     max=1
# #     min=-1
# #     temp = X_sim[0]
# #     X_std = (temp - temp.min()) / (temp.max() - temp.min())
# #     X_scaled = X_std * (max - min) + min
# #     X_scaled = np.expand_dims(X_scaled, axis = 0)  
# #     #X_scaled = X_sim
    

#     if step==0:
#         inspect_X = X_sim
#     else:
#         inspect_X = np.concatenate((inspect_X, X_sim), axis=0)
        
#     # predict if there will be a collision in the next time step
#     preds = rf.predict(X_sim)
#     env.render()
#     preds_list.append(int(preds[0]))  # indexing the element and converting the float to int

#     if int(reward) == 1:
#         has_crashed.append(1)
#     elif int(reward) == 0:
#         has_crashed.append(0)            
#     if done == True:
#         env.reset() # going to different episode if crash has occurred/ episode duration crossed/ vehicle out of bounds


# end = time.time()
# print("time taken for simulation time prediction: ", round((end - start),2), "seconds")

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 277.70it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 292.87it/s]

  0%|▏                                                                                | 3/1000 [00:00<03:22,  4.93it/s]

origin:  [-13.36363636 -11.81818182]
origin:  [-13.36363636 -11.81818182]
origin:  [-13.36363636 -11.81818182]



100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 319.89it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 342.84it/s]


origin:  [-13.36363636 -11.81818182]
origin:  [-13.36363636 -11.81818182]
origin:  [-13.36363636 -11.81818182]



100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 341.23it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 308.02it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is saf

...

origin:  [-13.36363636 -11.81818182]
origin:  [-13.36363636 -11.81818182]
origin:  [-13.36363636 -11.81818182]


C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 335.68it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details an

origin:  [-13.36363636 -11.81818182]
origin:  [-13.36363636 -11.81818182]



C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:34<00:00, 10.63it/s]

origin:  [-13.36363636 -11.81818182]
time taken for simulation time prediction:  94.06 seconds


In [8]:
history_size=2
X_sim = np.array([]) # contains all feature values
has_crashed = [] # contains crash (1) or no-crash (0) for each time step
preds_list = []

start = time.time()
for step in range(1000):
    action = None
    obs, reward, done, info = env.step(action)

    temp_obs = obs.flatten() # this returns a 1D numpy array [5, 1, 23, 4,...]
    temp_obs = np.expand_dims(temp_obs, axis = 0) # turning dataset into 2D; [[5, 1, 23, 4,...]]
    
    temp_obs = add_corner_position_vectors(pd.DataFrame(temp_obs))
    
    if step == 0:
        X_sim = temp_obs     
    else:
        X_sim = np.concatenate((X_sim, temp_obs), axis = 0)
    
    # concatenate 4 observations
    if step + 1 >= history_size:  # wait till there are atleast `history_size` observations
        X_concatenated = np.array([])
        for i in range(step + 1 - history_size, len(X_sim) - history_size + 1):   # this makes sure we get `history_size` amount of samples for each row in the final dataset             
            # saving `history_size` rows in temp_X
            for j in range(i, i + history_size):
                if j==i:  # the first iteration for each i           
                    temp_X = X_sim[j]
                else:
                    temp_X = np.append(temp_X, X_sim[j], axis = 0)
            #print(temp_X.shape)        
            # saving concatenated row in X_concatenated
            X_concatenated = np.concatenate((X_concatenated, temp_X.flatten()), axis = 0)
            X_concatenated = np.expand_dims(X_concatenated, axis = 0)
    
            # predict if there will be a collision in the next time step
            preds = rf.predict(X_concatenated)
            preds_list.append(int(preds[0]))  # indexing the element and converting the float to int

    if done == True:
        has_crashed.append(1)
        env.reset()
    else:
        has_crashed.append(0)

end = time.time()
print("time taken for simulation time prediction: ", round((end - start),2), "seconds")

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 419.44it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 634.13it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 437.76it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 416.70it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 463.77it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 383.01it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 538.52it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 500.43it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 408.78it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 491.02it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

...

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 523.32it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 519.79it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 300.46it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 437.66it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 581.61it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 411.86it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 454.74it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 390.44it/s]
C:\Users\mosta\anaconda3\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

time taken for simulation time prediction:  49.12 seconds


In [10]:
len(preds_list)

C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


999

In [6]:
# inspect_preds = np.expand_dims(np.array(preds_list), axis=0)
# inspect_y = np.expand_dims(np.array(has_crashed), axis = 0)
# dataset_inspect = np.concatenate((inspect_X, inspect_y.T), axis = 1)
# dataset_inspect = np.concatenate((dataset_inspect, inspect_preds.T), axis = 1)
# df_dataset_inspect = pd.DataFrame(dataset_inspect)
# display(df_dataset_inspect.head())

C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9,...,97,98,99,100,101,102,103,104,105,106
0,1.0,0.199995,0.150411,0.190780,0.109514,0.186553,0.147275,0.209210,0.191308,0.213437,...,-0.072026,0.278673,-0.067984,0.240562,0.061589,0.020149,0.914669,0.404204,1.0,1.0
1,1.0,0.086917,0.889284,0.077702,0.848387,0.073475,0.886148,0.096132,0.930181,0.100359,...,0.082521,-0.053978,0.084302,-0.094979,0.066137,-0.073917,0.984014,0.178089,0.0,0.0
2,1.0,0.094940,0.904878,0.085726,0.863981,0.081499,0.901742,0.104155,0.945774,0.108382,...,0.084285,-0.062191,0.086066,-0.103192,0.058516,-0.079780,0.984014,0.178089,0.0,0.0
3,1.0,0.103383,0.921285,0.094168,0.880388,0.089941,0.918149,0.112598,0.962182,0.116825,...,0.085845,-0.071056,0.087626,-0.112056,0.051742,-0.084992,0.984014,0.178089,0.0,0.0
4,1.0,0.112685,0.933127,0.102461,0.896629,0.099240,0.936075,0.122908,0.969625,0.126129,...,0.086738,-0.075210,0.088519,-0.116211,0.030413,-0.051541,0.984014,0.178089,0.0,0.0


In [11]:
def edit_test_images(df_dataset_inspect, testing_duration=1000):
    '''predictions is a list of the predictions at each time step. It's length is equal to testing_duration'''
    
    for i in range(testing_duration):
        img_filename = 'highway_env_saved_frames\highway-env_{}.png'.format(i)
        img = Image.open(img_filename)

        # Call draw Method to add 2D graphics in an image
        draw_object = ImageDraw.Draw(img)
        
        font = ImageFont.truetype("arial.ttf", 15)
        
        # Add Text to an image
        draw_object.text((440, 10), "v_x:" + str(round(df_dataset_inspect.iloc[i,11],3)), fill=(255, 255, 255), font=font)
        draw_object.text((440, 30), "v_y:" + str(round(df_dataset_inspect.iloc[i,12],3)), fill=(255, 255, 255), font=font)
        draw_object.text((400, 50), "ego_heading:" + str(round(np.arccos(np.array(df_dataset_inspect.iloc[i,5])),2)), fill=(255, 255, 255), font=font) 
        draw_object.text((430, 70), "prediction:" + str(df_dataset_inspect.iloc[i,-1]), fill=(255, 255, 255), font=font)
        
        img.save('highway_env_saved_frames\edited_images\edited_highway-env_{}.png'.format(i),"PNG")

<>:5: DeprecationWarning: invalid escape sequence \h
<>:19: DeprecationWarning: invalid escape sequence \e


In [8]:
# # naming all the columns; useful for the next step
# n_vehicles = 7
# col_name_list = []
# for i in range(n_vehicles):
#   col_name_list = col_name_list + ['presence_'+str(i),'x_'+str(i),'y_'+str(i),'vx_'+str(i),'vy_'+str(i),'cos_h_'+str(i),'sin_h_'+str(i)]
# col_name_list = col_name_list + ['actual_collision']
# col_name_list = col_name_list + ['predicted_collision']

# df_dataset_inspect.columns = col_name_list

In [9]:
# n_vehicles = 7
# for i in range(n_vehicles):
#   df_dataset_inspect.hist(figsize=(15,10), column=['presence_'+str(i),'x_'+str(i),'y_'+str(i),'vx_'+str(i),'vy_'+str(i),'cos_h_'+str(i),'sin_h_'+str(i)], range=(-1,1))

In [10]:
df_dataset_inspect.shape

(1000, 107)

In [11]:
print("Number of label=1: ", df_dataset_inspect[df_dataset_inspect.iloc[:,-1] == 1.0].shape[0])
print("Number of label=0: ", df_dataset_inspect[df_dataset_inspect.iloc[:,-1] == 0.0].shape[0])

Number of label=1:  212
Number of label=0:  788


In [12]:
history_size = 1
collision_label_list = has_crashed[history_size:] # there was no feature value for `history_size` number of values in has_crashed
preds_list = preds_list[:]  # there is no true value for the last sample's next step prediction

In [13]:
# start = time.time()
# edit_test_images(df_dataset_inspect)
# end = time.time()
# print("time taken: ", (end-start))

time taken:  34.02207708358765


In [13]:
print("Number of label=1 in collision_label_list: ", sum(collision_label_list))
print("Number of label=1 in preds_list: ", sum(preds_list))
print("Number of label=0 in collision_label_list: ", len(collision_label_list) - \
                                                     sum(collision_label_list))
print("Number of label=0 in preds_list: ", len(preds_list) - sum(preds_list))

Number of label=1 in collision_label_list:  229
Number of label=1 in preds_list:  274
Number of label=0 in collision_label_list:  770
Number of label=0 in preds_list:  725


In [14]:
#calculate accuracy
acc = accuracy_score(collision_label_list, preds_list)
acc_percent = acc*100
error_percent = 100 - acc_percent
print("Simulation time Error:", round(error_percent,2), "%")
print("Simulation time Accuracy:", round(acc_percent,2), "%")

Simulation time Error: 11.71 %
Simulation time Accuracy: 88.29 %


In [15]:
prec_score_1 = precision_score(collision_label_list, preds_list, pos_label = 1)
recall_score_1 =  recall_score(collision_label_list, preds_list, pos_label = 1)
print("Precision of label=1: ", prec_score_1)
print("Recall of label=1: ", recall_score_1)

Precision of label=1:  0.7043795620437956
Recall of label=1:  0.8427947598253275


C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
prec_score_0 = precision_score(collision_label_list, preds_list, pos_label = 0)
recall_score_0 =  recall_score(collision_label_list, preds_list, pos_label = 0)
print("Precision of label=0: ", prec_score_0)
print("Recall of label=0: ", recall_score_0)

Precision of label=0:  0.9503448275862069
Recall of label=0:  0.8948051948051948


C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
actual_y = np.array(collision_label_list)
predicted_y = np.array(preds_list)

wrong_index_save=[]   # contains indices of samples where actual and predict do not match
for idx, val in enumerate(actual_y == predicted_y):    # (actual_y == predicted_y) : returns a boolean numpy array
    if val==False:
        wrong_index_save.append(idx)

C:\Users\mosta\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
len(wrong_index_save)

61

In [20]:
inspect_X.shape

(1000, 105)

In [21]:
wrong_X = np.array([])   # contains all the samples where the model went wrong
for wrong_index in wrong_index_save:
    if len(wrong_X)==0:
        wrong_X = np.expand_dims(inspect_X[wrong_index], axis=0)
    else:
        wrong_X = np.concatenate((wrong_X, np.expand_dims(inspect_X[wrong_index], axis=0)), axis=0)

In [22]:
wrong_actual_y=[]
for wrong_index in wrong_index_save:
    wrong_actual_y.append(actual_y[wrong_index])

In [23]:
#wrong_index_save

In [24]:
for i in wrong_index_save:
        img_filename = 'highway_env_saved_frames\highway-env_{}.png'.format(i)
        img = Image.open(img_filename)

        # Call draw Method to add 2D graphics in an image
        draw_object = ImageDraw.Draw(img)
        
        font = ImageFont.truetype("arial.ttf", 15)
        
        # Add Text to an image
        draw_object.text((440, 10), "v_x:" + str(round(df_dataset_inspect.iloc[i,11],3)), fill=(255, 255, 255), font=font)
        draw_object.text((440, 30), "v_y:" + str(round(df_dataset_inspect.iloc[i,12],3)), fill=(255, 255, 255), font=font)
        draw_object.text((400, 50), "ego_heading:" + str(round(np.arccos(np.array(df_dataset_inspect.iloc[i,5])),2)), fill=(255, 255, 255), font=font) 
        draw_object.text((430, 70), "prediction:" + str(df_dataset_inspect.iloc[i,-1]), fill=(255, 255, 255), font=font)
        
        img.save('highway_env_saved_frames\wrong_only_edited_images\wrong_only_edited_highway-env_{}.png'.format(i),"PNG")

<>:2: DeprecationWarning: invalid escape sequence \h
<>:16: DeprecationWarning: invalid escape sequence \w


In [22]:
file = open("results.txt", "a")
file.write("Dataset: mini_new_dataset_w_corner_position_vectors.csv\n")
file.write("Model: predict_current_time_step_corner_position_vectors_added_rf_100.joblib \n")
file.write("Additional Comments: no corner points added \n")
file.write("Number of label=1 in collision_label_list:  {0} \nNumber of label=1 in preds_list: \
           {1}   \nNumber of label=0 in collision_label_list:  {2}  \nNumber of label=0 in preds_list: \
           {3}\n".format(sum(collision_label_list), sum(preds_list), len(collision_label_list) - \
           sum(collision_label_list), len(preds_list) - sum(preds_list)))

file.write("Simulation time Accuracy: {}% \n".format(round(acc_percent,3)))
file.write("Precision of label=1:     {} \n".format(round(prec_score_1,3)))
file.write("Recall of label=1:        {} \n".format(round(recall_score_1,3)))
file.write("Precision of label=0:     {} \n".format(round(prec_score_0,3)))
file.write("Recall of label=0:        {} \n".format(round(recall_score_0,3)))
file.write("----------------------------------------end------------------------------------------------\n")
file.close()

### ANN

In [ ]:
# class ANNlearner(nn.Module):
#   def __init__(self,input_size, output_size, learning_rate):
#     super().__init__()
#     neurons1 = 64
#     neurons2 = 128
#     neurons3 = 256
#     neurons4 = 128
#     neurons5 = 64
#     self.h1 = nn.Linear(in_features=input_size, out_features=neurons1) 
#     self.h2 = nn.Linear(in_features=neurons1, out_features=neurons2)
#     self.h3 = nn.Linear(in_features=neurons2, out_features=neurons3)
#     self.h4 = nn.Linear(in_features=neurons3, out_features=neurons4)
#     self.h5 = nn.Linear(in_features=neurons4, out_features=neurons5)
#     self.out = nn.Linear(in_features=neurons5, out_features=output_size)
#     self.optimizer = optim.Adam(self.parameters(), lr = learning_rate)
#     self.sigmoid = nn.Sigmoid()
  
#   def forward(self, t):
#     t = F.relu(self.h1(t))
#     t = F.relu(self.h2(t))
#     t = F.relu(self.h3(t))
#     t = F.relu(self.h4(t))
#     t = F.relu(self.h5(t))
#     t = self.out(t)
#     out = self.sigmoid(t)
#     return out

In [ ]:
class ANNlearner(nn.Module):
  def __init__(self,input_size, output_size, learning_rate):
    super().__init__()
    neurons1 = 128
    neurons2 = 64
    self.h1 = nn.Linear(in_features=input_size, out_features=neurons1) 
    self.h2 = nn.Linear(in_features=neurons1, out_features=neurons2)
    self.out = nn.Linear(in_features=neurons2, out_features=output_size)
    self.optimizer = optim.Adam(self.parameters(), lr = learning_rate)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, t):
    t = F.relu(self.h1(t))
    t = F.relu(self.h2(t))
    t = self.out(t)
    out = self.sigmoid(t)
    return out

In [ ]:
# model training parameters
learning_rate = 0.001
num_of_epochs = 50
input_size = 49 #X.shape[1] # all the feature columns
output_size = 1
mini_batch_size = 2000

# model
ann = ANNlearner(input_size, output_size, learning_rate)
ann.load_state_dict(torch.load("collision_datasets_see_behind_true_absolute_false/predict_current_time_step/ann_w_length_width_wo_vx_vy"))

In [ ]:
# loading env
env = RandomFlowEnv()
env.config['vehicles_count'] = 40
env.configure({
    "observation": {
        'type': 'Kinematics',
        "features": ["presence", "x", "y", "cos_h", "sin_h"],
        'vehicles_count': 7
    }
})
env.reset()
print(env.config)

In [ ]:
env.observation_type.see_behind

In [ ]:
X_sim = np.array([]) # contains all feature values
has_crashed = [] # contains crash (1) or no-crash (0) for each time step
preds_list = []
inspect_X = np.array([])

start = time.time()
for step in tqdm(range(1000)):
    action = None
    obs, reward, done, info = env.step(action)

    temp_obs = obs.flatten() # this returns a 1D numpy array [5, 1, 23, 4,...]
    X_sim = np.expand_dims(temp_obs, axis = 0) # turning dataset into 2D; [[5, 1, 23, 4,...]] 
    X_sim = inserting_col(X_sim)
    
    if step==0:
        inspect_X = X_sim
    else:
        inspect_X = np.concatenate((inspect_X, X_sim), axis=0)
    
    # extra processing for ANN learner
    X_sim = X_sim.astype('float32', copy=False)
    X_tensor = torch.from_numpy(X_sim)
    # predict if there will be a collision in the next time step
    preds = ann(X_tensor)
    
    if preds.item() >= 0.5:
        preds_list.append(1)  # indexing the element and converting the float to int
    elif preds.item() < 0.5:
        preds_list.append(0)
        
    if int(reward) == 1:
        has_crashed.append(1)
    elif int(reward) == 0:
        has_crashed.append(0)            
    if done == True:
        env.reset() # going to different episode if crash has occurred/ episode duration crossed/ vehicle out of bounds


end = time.time()
print("time taken for simulation time prediction: ", round((end - start),2), "seconds")

In [ ]:
inspect_preds = np.expand_dims(np.array(preds_list), axis=0)
inspect_y = np.expand_dims(np.array(has_crashed), axis = 0)
dataset_inspect = np.concatenate((inspect_X, inspect_y.T), axis = 1)
dataset_inspect = np.concatenate((dataset_inspect, inspect_preds.T), axis = 1)
df_dataset_inspect = pd.DataFrame(dataset_inspect)
display(df_dataset_inspect.head())

In [ ]:
df_dataset_inspect.shape

In [ ]:
#df_dataset_inspect.to_csv("collision_datasets_see_behind_true_absolute_false/predict_current_time_step/rf_100_inspect_combined_dataset_predict_current_step_with_50_surrounding_vehicles_in_testing.csv")

In [ ]:
print("Number of label=1: ", df_dataset_inspect[df_dataset_inspect.iloc[:,-1] == 1.0].shape[0])
print("Number of label=0: ", df_dataset_inspect[df_dataset_inspect.iloc[:,-1] == 0.0].shape[0])

In [ ]:
history_size = 1
collision_label_list = has_crashed[:] # there was no feature value for `history_size` number of values in has_crashed
preds_list = preds_list[:]  # there is no true value for the last sample's next step prediction

In [ ]:
print("Number of label=1 in collision_label_list: ", sum(collision_label_list))
print("Number of label=1 in preds_list: ", sum(preds_list))
print("Number of label=0 in collision_label_list: ", len(collision_label_list) - \
                                                     sum(collision_label_list))
print("Number of label=0 in preds_list: ", len(preds_list) - sum(preds_list))

In [ ]:
#calculate accuracy
acc = accuracy_score(collision_label_list, preds_list)
acc_percent = acc*100
error_percent = 100 - acc_percent
print("Simulation time Error:", round(error_percent,2), "%")
print("Simulation time Accuracy:", round(acc_percent,2), "%")

In [ ]:
prec_score_1 = precision_score(collision_label_list, preds_list, pos_label = 1)
recall_score_1 =  recall_score(collision_label_list, preds_list, pos_label = 1)
print("Precision of label=1: ", prec_score_1)
print("Recall of label=1: ", recall_score_1)

In [ ]:
prec_score_0 = precision_score(collision_label_list, preds_list, pos_label = 0)
recall_score_0 =  recall_score(collision_label_list, preds_list, pos_label = 0)
print("Precision of label=0: ", prec_score_0)
print("Recall of label=0: ", recall_score_0)